In [1]:
pip install elasticsearch==7.13.1

     |████████████████████████████████| 354 kB 8.2 MB/s 


### Import packages

In [2]:
from elasticsearch import Elasticsearch

In [3]:
%%bash

wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512
tar -xzf elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
sudo chown -R daemon:daemon elasticsearch-7.9.2/
shasum -a 512 -c elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512 

elasticsearch-oss-7.9.2-linux-x86_64.tar.gz: OK


Run the instance as a daemon process

In [4]:
%%bash --bg

sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch

Starting job # 0 in a separate thread.


Once the instance has been started, grep for `elasticsearch` in the processes list to confirm the availability.

In [5]:
%%bash

ps -ef | grep elasticsearch

root         161     159  0 06:05 ?        00:00:00 sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch
daemon       162     161  0 06:05 ?        00:00:00 /bin/bash elasticsearch-7.9.2/bin/elasticsearch
root         168     163  0 06:05 ?        00:00:00 grep elasticsearch
daemon       187     162  0 06:05 ?        00:00:00 /bin/bash elasticsearch-7.9.2/bin/elasticsearch
daemon       188     187  0 06:05 ?        00:00:00 /content/elasticsearch-7.9.2/jdk/bin/java -version


query the base endpoint to retrieve information about the cluster.

In [6]:
import time
time.sleep(10)

In [10]:
%%bash

curl -sX GET "localhost:9200/"

{
  "name" : "1393344e7b65",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "Io1q9W-dQkKwIASAJPiQLQ",
  "version" : {
    "number" : "7.9.2",
    "build_flavor" : "oss",
    "build_type" : "tar",
    "build_hash" : "d34da0ea4a966c4e49417f2da2f244e3e97b4e6e",
    "build_date" : "2020-09-23T00:45:33.626720Z",
    "build_snapshot" : false,
    "lucene_version" : "8.6.2",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [11]:
es = Elasticsearch(hosts = ["http://localhost:9200"])

In [12]:
INDEX = 'wiki'
if not es.indices.exists(INDEX):
		es.indices.create(index=INDEX)

In [13]:
!curl -XGET localhost:9200/_cat/indices?v
# es.indices.delete(index=INDEX)

health status index uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   wiki  XhDXP_3-SveYq1_R3JuvyA   1   1          0            0       208b           208b


In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
import json
with open('/content/drive/MyDrive/ColabNotebooks/wikipedia_documents.json', 'r', encoding='utf-8') as file:
    wiki = json.load(file)

In [16]:
for idx, text in wiki.items():
    body = {'text': text['text']}
    es.index(index=INDEX, doc_type='news', id=int(idx)+1, body=body)

/usr/local/lib/python3.7/dist-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: [types removal] Specifying types in document index requests is deprecated, use the typeless endpoints instead (/{index}/_doc/{id}, /{index}/_doc, or /{index}/_create/{id}).
  warnings.warn(message, category=ElasticsearchWarning)


In [20]:
with open('/content/drive/MyDrive/ColabNotebooks/train.json', 'r', encoding='utf-8') as file:
    train = json.load(file)


'대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?'

In [59]:
print(train['question'][5])
print(train['context'][5])

카드모스의 부하들이 간 곳에는 무엇이 있었는가?
델포이의 신탁에 따라 암소를 따라간 카드모스는 테베 땅에 이르렀다. 카드모스는 암소를 잡아서 신들에게 공양하려고 부하들에게 근처의 샘으로 물심부름을 보냈다. 샘은 드래곤이 지키고 있었고, 드래곤은 카드모스의 부하 여럿을 죽인 뒤 카드모스의 칼에 죽었다.\n\n《비블리오테카》에 따르면 이 드래곤은 아레스의 신수였다고 한다. 아테나는 드래곤의 이빨 중 절반을 카드모스에게 주고 그것을 땅에 심으라고 했다. 카드모스가 그렇게 하자 고랑마다 사나운 무장한 사내들이 튀어나왔다. 그들에게 겁을 먹은 카드모스는 그들 사이에 돌을 집어던졌고, 돌을 누가 던졌냐고 시비가 붙은 용아병들은 서로 싸우다가 다섯 명만 남기고 나머지는 모두 죽었다. 살아남은 용아병 다섯은 에키온, 우다에오스, 크토노니오스, 퓌헤레노르, 펠로루스이며, 이 다섯은 카드모스를 도와 테베라는 도시를 건립했다. 카드모스는 드래곤을 죽인 대가로 8년동안 아레스의 노예로 살았고, 그 기간이 끝나자 아레스와 아프로디테의 딸인 하르모니아를 아내로 맞았다. \n\n한편, 미틸레네의 헬라니코스의 판본에 따르면 애초부터 용아병은 다섯 명이 튀어나왔으며, 아레스가 카드모스를 죽이려고 하는 것을 제우스가 나서서 살려 주었다. 용아병들 중 에키온은 뒤에 카드모스의 딸 아가베와 결혼했고, 둘 사이에 태어난 아들 펜테우스가 카드모스의 뒤를 이어 테베의 왕이 되었다.


In [60]:
body = {'query': {'match': {'text': train['question'][5]}}, 'size': 40}
# {'took': 109, 
# 'timed_out': False, 
# '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 
# 'hits': {'total': {'value': 1738, 'relation': 'eq'}, 
#          'max_score': 29.56596, 
#          'hits': [{'_index': 'wiki', '_type': 'news', '_id': '52397', '_score': 29.56596, '_source': {'text': '~'}}, 
#                   ...]}}
res = es.search(index=INDEX, body=body)

In [61]:
count = 0
for a in res['hits']['hits']:
    print(count)
    print(a)
    
    count += 1

0
{'_index': 'wiki', '_type': 'news', '_id': '37483', '_score': 18.393103, '_source': {'text': '델포이의 신탁에 따라 암소를 따라간 카드모스는 테베 땅에 이르렀다. 카드모스는 암소를 잡아서 신들에게 공양하려고 부하들에게 근처의 샘으로 물심부름을 보냈다. 샘은 드래곤이 지키고 있었고, 드래곤은 카드모스의 부하 여럿을 죽인 뒤 카드모스의 칼에 죽었다.\\n\\n《비블리오테카》에 따르면 이 드래곤은 아레스의 신수였다고 한다. 아테나는 드래곤의 이빨 중 절반을 카드모스에게 주고 그것을 땅에 심으라고 했다. 카드모스가 그렇게 하자 고랑마다 사나운 무장한 사내들이 튀어나왔다. 그들에게 겁을 먹은 카드모스는 그들 사이에 돌을 집어던졌고, 돌을 누가 던졌냐고 시비가 붙은 용아병들은 서로 싸우다가 다섯 명만 남기고 나머지는 모두 죽었다. 살아남은 용아병 다섯은 에키온, 우다에오스, 크토노니오스, 퓌헤레노르, 펠로루스이며, 이 다섯은 카드모스를 도와 테베라는 도시를 건립했다. 카드모스는 드래곤을 죽인 대가로 8년동안 아레스의 노예로 살았고, 그 기간이 끝나자 아레스와 아프로디테의 딸인 하르모니아를 아내로 맞았다. \\n\\n한편, 미틸레네의 헬라니코스의 판본에 따르면 애초부터 용아병은 다섯 명이 튀어나왔으며, 아레스가 카드모스를 죽이려고 하는 것을 제우스가 나서서 살려 주었다. 용아병들 중 에키온은 뒤에 카드모스의 딸 아가베와 결혼했고, 둘 사이에 태어난 아들 펜테우스가 카드모스의 뒤를 이어 테베의 왕이 되었다.'}}
1
{'_index': 'wiki', '_type': 'news', '_id': '31153', '_score': 12.442965, '_source': {'text': '464년, 아버지인 효무제가 죽어서 황제로 즉위하였다.\n\n어머니인 왕태후가 병으로 위독하여 유자업을 불렀는데도 "환자가 있는 곳에는 귀신이 나온다." 라고 말하여 문병에도 가지 않았다. 그러자 왕태후가 노하여 시종에게